# Project Tracking in TaskJuggler

It is one thing to have created a plan for your project, it is another to continue to engage with the 
plan to evaluate how well execution matches the plan. This can be accomplished using a number of tools
within TaskJuggler:

1. *Scenarios* ([1](http://taskjuggler.org/tj3/manual/Day_To_Day_Juggling.html#Working_with_multiple_scenarios), [2](http://taskjuggler.org/tj3/manual/scenario.html)) - an [example](Sample-Files/foundation-04.tjp) of which we saw in the comparison of duration- and 
effort-based plans. 
2. Noting *completion* ([1](http://taskjuggler.org/tj3/manual/Day_To_Day_Juggling.html#Using_completion_values), [2](http://taskjuggler.org/tj3/manual/complete.html)) - by adding attributes to tasks that reflect the degree of completion by the current date. **Completion information *does not* change scheduling calculations**. 
3. By using TaskJuggler's integrated [status tracking system](http://taskjuggler.org/tj3/manual/Day_To_Day_Juggling.html#Implementing_the_status_tracking_system) - supporting time tracking, effort reporting, and progress reporting through a combinatino of email and web-based tools. This very powerful capability is beyond the scope of today's workshop. 

In this demonstration and exercise we will work with alternative scenarios (*planned* and *actual*) to see how our project execution aligns with our plan. Within those scenarios we will also see an example of manually adding completion information. 

[foundation-05.tjp file](Sample-Files/foundation-05.tjp)

    project foundation "Foundation Project" 2018-07-01 - 2019-12-31 {
        currency "USD"
        timeformat "%Y-%m-%d"
        numberformat "-" "" "," "." 1
        currencyformat "(" ")" "," "." 0
        now 2018-11-01-00:00
        weekstartsmonday
        workinghours mon - fri 9:00 - 12:00, 13:00 - 18:00
        workinghours sat, sun off
        scenario planned "Initial Plan" {
            scenario actual "Actual Plan Execution" {
                scenario planb "Plan B"
            }
        }
    }
    
    
    ############## accounts ################
    
    account cost "Project Cost" {
      account personnel "Personnel"
      account materials "Materials"
    }
    account rev "Payments"
    
    balance cost rev
    
    ############## resources ###############
    resource boss "The Big Boss" {
        efficiency .9
        workinghours mon - fri 7:00 - 12:00, 13:00 - 18:00
        workinghours sat, sun 10:00 - 15:00
        rate 300
    }
    
    resource painter "The painter" {
        rate 200
        limits {dailymax 4h}
    }
    
    resource crew "The crew" {
        rate 175
        resource pj "PJ"
        resource jb "JB"
        resource ay "AY" {
            rate 250
            actual:leaves special 2018-10-01 +3m
        }
    }
    
    resource newcrew "Additional Crew Members" {
        rate 200
        resource ng1 "NG 1"
        resource ng2 "NG 2"
        resource ng3 "NG 3"
    }
    
    ############## tasks ###################
    task projectstart "Project Start" {
        start ${projectstart}
    }
    
    
    task complete "Complete Project"{
    task doing "Making the Goods" {
        start ${projectstart}
        task buy_materials "Buy the materials" {
            planned:duration 1m
            actual:duration 2w
            allocate ay
            limits {maximum 8h}
            chargeset personnel
        }
        task glue_together "Glue everything together" {
            depends !buy_materials # relative reference to task
            planned:effort 8w
            actual:effort 12w
            allocate crew
            chargeset personnel
        }
        task cleanup "Clean up the mess" {
            depends complete.doing.glue_together # absolute reference to task
            planned:effort 30d
            actual:effort 60d
            allocate crew
            planb:allocate newcrew
            chargeset personnel
        }
    }
    
    task refining "Refining the Goods" {
        depends complete.doing
        task paint "Paint the items" {
            planned:effort 3w
            actual:effort 5w
            actual:complete 25
            allocate painter
            chargeset personnel
        }
        task attach_bells "Screw bells onto items" {
            depends !paint
            task buy_bells "Buy bells"{
                planned:duration 1w
                planb:duration 1m
                chargeset materials
                charge 2000 onstart
            }
            task use_screwdriver "Use screwdriver" {
                depends !buy_bells
                planned:effort 1m
                planb:effort 2m
                allocate crew
                planb:allocate newcrew
                chargeset personnel
            }
        }
        task repaint "Repaint the items" {
            depends !attach_bells
            planned:effort 2m
            planb:effort 3m
            allocate painter
            chargeset personnel
        }
        task explain "Explain to Boss what you spent the last 3 months doing" {
            depends !repaint
            effort 2h
            allocate ay
            chargeset personnel
        }
    }
    
    task selling "Selling the Goods" {
        depends complete.refining.explain
        planned:effort 4m
        planb:effort 4m
        allocate boss
        chargeset personnel
    }
    
    task get_paid "Get paid for the goods" {
        depends complete.selling
        chargeset rev
        planned:charge 80000 onend
    }
    }
    
    ############## reports #################
    
    taskreport "reports/foundation-05_taskreport" {
        formats html, csv
        columns bsi, name, scenario, start, end, effort, cost, revenue, chart
        hideresource 0
        scenarios planned, actual, planb
        headline "Project Breakdown"
    }
    



In [34]:
%%bash
cd Sample-Files
tj3 foundation-05.tjp

TaskJuggler v3.6.0 - A Project Management Software

Copyright (c) 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016
              by Chris Schlaeger <cs@taskjuggler.org>

This program is free software; you can redistribute it and/or modify it under
the terms of version 2 of the GNU General Public License as published by the
Free Software Foundation.

Reading file foundation-05.tjp                               [      Done      ]
Preparing scenario Initial Plan                              [      Done      ]
Scheduling scenario Initial Plan                             [      Done      ]
Checking scenario Initial Plan                               [      Done      ]
Preparing scenario Actual Plan Execution                     [      Done      ]
Scheduling scenario Actual Plan Execution                    [      Done      ]
Checking scenario Actual Plan Execution                      [      Done      ]
Preparing scenario Plan B                                    [      Done

producing this HTML report:

[Sample-Files/reports/foundation-05_taskreport.html](Sample-Files/reports/foundation-05_taskreport.html)

and the following CSV file:

[Sample-Files/reports/foundation-05_taskreport](Sample-Files/reports/foundation-05_taskreport)

## Practice ...

Update your developing plan with some of the project tracking and/or alternative scenario elements outlined above. When done you can generate 
new HTML and CSV reports that illustrate/contain the results of the project scheduling processs
in TaskJuggler.  

### Activity:

1. Add a scenario or two to your project plan that reflects the ongoing evolution of you plan as it is executed.

2. Run the TaskJuggler scheduler to test your skeleton to make sure that it does not generate any errors. If it does, see if you can fix them and re-run the scheduler. 

<video controls src="images/Timer10Minutestory.mov" />

-------------------
[(0)](TJ-00%20What%20is%20TaskJuggler.ipynb) -- 
[(1)](TJ-01%20Project%20Skeleton.ipynb) -- 
[(2)](TJ-02%20A%20Fleshed%20Out%20TaskJuggler%20Outline.ipynb) -- 
[(3)](TJ-03%20Progressive%20Elaboration%20of%20Tasks%20in%20TaskJuggler.ipynb) -- 
[(4)](TJ-04%20Assigning%20Resources%20%26%20Cost%20Estimation%20in%20TaskJuggler.ipynb) --
(5) -- 
[(6)](TJ-06%20Visualization%20%26%20Reporting%20in%20TaskJuggler.ipynb)